In [2]:
import pandas as pd
import numpy as np
import datetime
import warnings

In [26]:
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore", category=DeprecationWarning)
pd.set_option('future.no_silent_downcasting', True)

In [4]:
date_list = pd.read_excel('date_list.xlsx')
date_list = date_list.drop(columns='Unnamed: 0')
date_list = list(date_list['Date'])

In [5]:
bond_identity = pd.read_csv('bond_msbond.csv')
bond_price_1 = pd.read_csv('Bond_TrBei_to_20240816.csv')
bond_price_2 = pd.read_csv('Bond_TrBei_from_20240819_to_20250226.csv')
bond_price = pd.concat([bond_price_1,bond_price_2])
bond = bond_price.merge(bond_identity,on='portId',how='left')
bond['issuedyear'] = bond['issueddate'].str[:4]
bond['issuedyear'] = bond['issuedyear'].astype('int')
bond = bond.drop(columns='txtNo')

C:\Users\Infovesta PC-04\AppData\Local\Temp\ipykernel_11740\3437434067.py:2: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  bond_price_1 = pd.read_csv('Bond_TrBei_to_20240816.csv')


In [6]:
bond['portReleased'] = bond['portReleased'].replace(r'\N',np.nan)
bond['jtdatedt'] = pd.to_datetime(bond['jtdate'])
bond['portReleaseddt'] = pd.to_datetime(bond['portReleased'])
bond['tenorday'] = bond['jtdatedt'] - bond['portReleaseddt']
bond['tenorday'] = pd.to_numeric(bond['tenorday'].astype('str').str[:-5],errors='coerce')
bond['tenoryear'] = bond['tenorday']/365
bond = bond.drop(columns=['jtdatedt','portReleaseddt'])

In [31]:
bond

,issueddate,portId,portrank,portrank2,highPrice,loPrice,lastPrice,frekuensi,volumeTrans,valueTrans,WAPPrice,coupon,YTMValue,MDValue,portId2,portName,portType,syariah,dollar,outstanding,jtdate,portissuer,portReleased,periodekupon,flag,issuedyear,tenorday,tenoryear,tradeday,weekend,USDIDR,nominal,interest,accruedinterest,marketvalue
0,2009-06-05,ADHI04,idA-,A,96.50,95.00,95.55,0.0,4.000,3.82320,95.58,11.00,12.751,2.518285852,ADHI04,Obligasi IV Adhi Tahun 2007,B,N,N,3.750000e+11,2012-07-06,ADHI,2007-07-09,4.0,1.0,2009.0,1824.0,4.997260,Y,N,10039.0,3.750000e+11,1.130137e+08,1.130137e+08,3.584255e+11
1,2009-06-06,ADHI04,idA-,A,96.50,95.00,95.55,0.0,4.000,3.82320,95.58,11.00,12.751,2.518285852,ADHI04,Obligasi IV Adhi Tahun 2007,B,N,N,3.750000e+11,2012-07-06,ADHI,2007-07-09,4.0,1.0,2009.0,1824.0,4.997260,N,Y,10039.0,3.750000e+11,1.130137e+08,2.260274e+08,3.585385e+11
2,2009-06-07,ADHI04,idA-,A,96.50,95.00,95.55,0.0,4.000,3.82320,95.58,11.00,12.751,2.518285852,ADHI04,Obligasi IV Adhi Tahun 2007,B,N,N,3.750000e+11,2012-07-06,ADHI,2007-07-09,4.0,1.0,2009.0,1824.0,4.997260,N,Y,10039.0,3.750000e+11,1.130137e+08,3.390411e+08,3.586515e+11
3,2009-06-08,ADHI04,idA-,A,96.50,95.00,95.55,0.0,4.000,3.82320,95.58,11.00,12.751,2.518285852,ADHI04,Obligasi IV Adhi Tahun 2007,B,N,N,3.750000e+11,2012-07-06,ADHI,2007-07-09,4.0,1.0,2009.0,1824.0,4.997260,N,N,9985.0,3.750000e+11,1.130137e+08,4.520548e+08,3.587646e+11
4,2009-06-09,ADHI04,idA-,A,105.06,105.03,105.03,0.0,10.000,10.50500,105.05,11.00,9.0964,2.557353989,ADHI04,Obligasi IV Adhi Tahun 2007,B,N,N,3.750000e+11,2012-07-06,ADHI,2007-07-09,4.0,1.0,2009.0,1824.0,4.997260,Y,N,10076.0,3.750000e+11,1.130137e+08,5.650685e+08,3.944276e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3240380,2025-02-25,MPMF01BCN2,AA(idn),AA,100.09,100.07,100.09,2.0,2.000,2.00180,100.09,7.00,6.933268,1.37429,MPMF01BCN2,Obligasi Berkelanjutan I JACCS MPM Finance Ind...,B,N,N,2.300000e+11,2026-08-11,MPMF,2023-08-14,4.0,1.0,2025.0,1093.0,2.994521,Y,N,16303.0,2.300000e+11,4.410959e+07,4.410959e+07,2.302511e+11
3240381,2025-02-26,MPMF01BCN2,AA(idn),AA,100.09,100.07,100.09,2.0,2.000,2.00180,100.09,7.00,6.933268,1.37429,MPMF01BCN2,Obligasi Berkelanjutan I JACCS MPM Finance Ind...,B,N,N,2.300000e+11,2026-08-11,MPMF,2023-08-14,4.0,1.0,2025.0,1093.0,2.994521,N,N,16316.0,2.300000e+11,4.410959e+07,8.821918e+07,2.302952e+11
3240382,2025-02-26,MDKA05BCN2,idA+,A,103.00,100.50,103.00,115.0,65.715,67.68645,103.00,8.75,7.620863,2.623921,MDKA05BCN2,Obligasi Berkelanjutan V Merdeka Copper Gold T...,B,N,N,1.944000e+12,2028-02-25,MDKA,2025-02-26,4.0,1.0,2025.0,1094.0,2.997260,Y,N,16316.0,1.944000e+12,4.660274e+08,4.660274e+08,2.002786e+12
3240383,2025-02-26,BWPT01ACN2,NaN,NaN,100.25,99.75,100.25,14.0,21.280,21.33320,100.25,9.75,9.488908,0.94732,BWPT01ACN2,Obligasi Berkelanjutan I Eagle High Plantation...,B,N,N,2.392000e+10,2026-03-06,BWPT,2025-02-27,4.0,1.0,2025.0,372.0,1.019178,Y,N,16316.0,2.392000e+10,6.389589e+06,6.389589e+06,2.398619e+10


In [7]:
cols = list(bond.columns)
bond['tradeday'] = 'Y'
bond_i_list = []
for i in bond['portId'].unique():
    bond_i = bond[bond['portId']==i]
    start_date = bond_i['portReleased'].unique()[0]
    end_date = max(bond['issueddate'].unique())
    jt_date = bond_i['jtdate'].unique()[0]
    if type(jt_date) is str:
        if end_date>jt_date:
            end_date = jt_date
        else:
            pass
    else:
        print(f'JT date for {i} is not found')
        continue
    if type(start_date) is str:
        bond_i_date = pd.DataFrame({'issueddate':pd.date_range(start=start_date,end=end_date,freq='D')})
        if len(bond_i_date)>0:
            bond_i_date['issueddate'] = bond_i_date['issueddate'].astype('str')
            bond_i = bond_i_date.merge(bond_i,on='issueddate',how='left')
            bond_i = bond_i.sort_values(by='issueddate')
            bond_i.loc[:,cols] = bond_i.loc[:,cols].ffill()
            bond_i_list.append(bond_i)
        else:
            bond_i_list.append(bond_i)
    else:
        print(f'Released date for {i} is not found')
        continue
bond = pd.concat(bond_i_list)
bond = bond.dropna(subset='portId')
bond = bond.reset_index(drop=True)
bond['tradeday'] = bond['tradeday'].fillna('N')
bond['weekend'] = np.where((pd.to_datetime(bond['issueddate'], errors='coerce')).dt.dayofweek<5,'N','Y')



Released date for BBNI01XXBFTW is not found
Released date for BBTN11 is not found
Released date for BEXI03C is not found
Released date for BFNC02 is not found
Released date for BJBR05 is not found
Released date for FR0021 is not found
Released date for FR0022 is not found
Released date for FR0023 is not found
Released date for FR0025 is not found
Released date for FR0026 is not found
Released date for FR0027 is not found
Released date for FR0033 is not found
Released date for FR0049 is not found
Released date for GBRB0017NvBF is not found
Released date for GBRB0019NvBF is not found
Released date for GBRB0020NvBF is not found
Released date for IFR0001 is not found
Released date for INDF04 is not found
Released date for ISAT04A is not found
Released date for ISAT06A is not found
Released date for ORI003 is not found
Released date for ORI004 is not found
Released date for ORI005 is not found
Released date for ORI006 is not found
Released date for OTMA06C is not found
Released date for OTM

C:\Users\Infovesta PC-04\AppData\Local\Temp\ipykernel_11740\2691501162.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bond_i.loc[:,cols] = bond_i.loc[:,cols].ffill()
C:\Users\Infovesta PC-04\AppData\Local\Temp\ipykernel_11740\2691501162.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bond_i.loc[:,cols] = bond_i.loc[:,cols].ffill()


Released date for BEXI01ACN2 is not found


C:\Users\Infovesta PC-04\AppData\Local\Temp\ipykernel_11740\2691501162.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bond_i.loc[:,cols] = bond_i.loc[:,cols].ffill()
C:\Users\Infovesta PC-04\AppData\Local\Temp\ipykernel_11740\2691501162.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bond_i.loc[:,cols] = bond_i.loc[:,cols].ffill()
C:\Users\Infovesta PC-04\AppData\Local\Temp\ipykernel_11740\2691501162.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.i

Released date for SPN12160107 is not found


...

C:\Users\Infovesta PC-04\AppData\Local\Temp\ipykernel_11740\2691501162.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bond_i.loc[:,cols] = bond_i.loc[:,cols].ffill()
C:\Users\Infovesta PC-04\AppData\Local\Temp\ipykernel_11740\2691501162.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bond_i.loc[:,cols] = bond_i.loc[:,cols].ffill()
C:\Users\Infovesta PC-04\AppData\Local\Temp\ipykernel_11740\2691501162.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.i

Released date for BCAF02BCN2 is not found


C:\Users\Infovesta PC-04\AppData\Local\Temp\ipykernel_11740\2691501162.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bond_i.loc[:,cols] = bond_i.loc[:,cols].ffill()
C:\Users\Infovesta PC-04\AppData\Local\Temp\ipykernel_11740\2691501162.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bond_i.loc[:,cols] = bond_i.loc[:,cols].ffill()
C:\Users\Infovesta PC-04\AppData\Local\Temp\ipykernel_11740\2691501162.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.i

JT date for SPNS03122021 is not found


C:\Users\Infovesta PC-04\AppData\Local\Temp\ipykernel_11740\2691501162.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bond_i.loc[:,cols] = bond_i.loc[:,cols].ffill()
C:\Users\Infovesta PC-04\AppData\Local\Temp\ipykernel_11740\2691501162.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bond_i.loc[:,cols] = bond_i.loc[:,cols].ffill()
C:\Users\Infovesta PC-04\AppData\Local\Temp\ipykernel_11740\2691501162.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.i

In [8]:
#Tambah kurs BI
kurs_BI = pd.read_excel('kurs_BI.xls')
kurs_BI.columns = kurs_BI.loc[0]
kurs_BI = kurs_BI[1:].reset_index(drop=True)
kurs_BI = kurs_BI.rename(columns={'Nilai': 'USDIDR', 'Tanggal': 'issueddate'})
kurs_BI['issueddate'] = kurs_BI['issueddate'].apply(lambda x: x.strftime('%Y-%m-%d'))
bond = pd.merge(bond, kurs_BI[['issueddate', 'USDIDR']], on='issueddate', how='left')
bond_list = []
for i in bond['portId'].unique():
    bond_i = bond[bond['portId']==i]
    bond_i = bond_i.sort_values(by='issueddate')
    bond_i['USDIDR'] = bond_i['USDIDR'].ffill()
    bond_list.append(bond_i)
bond = pd.concat(bond_list)
bond = bond.reset_index(drop=True)
bond['outstanding'] = np.where(bond['dollar']=='Y',bond['outstanding']*bond['USDIDR'],bond['outstanding'])

C:\Users\Infovesta PC-04\AppData\Local\Temp\ipykernel_11740\862688011.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bond_i['USDIDR'] = bond_i['USDIDR'].ffill()


In [9]:
#Tambah nominal
bond_nominal = pd.read_csv('bond_msnominal.csv').drop('txtno',axis=1).rename(columns={'portid': 'portId','tanggal': 'issueddate'})
bond_nominal['portId + issueddate'] = bond_nominal['portId'].astype(str) + ' ' + bond_nominal['issueddate'].astype(str)
bond['portId + issueddate'] = bond['portId'].astype(str) + ' ' + bond['issueddate'].astype(str)
bond = bond.merge(bond_nominal[['portId + issueddate','nominal']],how='left',on='portId + issueddate')
bond = bond.drop(columns='portId + issueddate')
bond_list = []
for i in bond['portId'].unique():
    bond_i = bond[bond['portId']==i]
    bond_i = bond_i.sort_values(by='issueddate')
    outstanding_i = bond_i['outstanding'].dropna().unique()[0]
    bond_i['nominal'] = bond_i['nominal'].ffill()
    bond_i['nominal'] = bond_i['nominal'].fillna(outstanding_i)
    bond_list.append(bond_i)
bond = pd.concat(bond_list)
bond = bond.reset_index(drop=True)


In [30]:
bond[['portId', 'lastPrice']]

,portId,lastPrice
0,ADHI04,95.55
1,ADHI04,95.55
2,ADHI04,95.55
3,ADHI04,95.55
4,ADHI04,105.03
...,...,...
3240380,MPMF01BCN2,100.09
3240381,MPMF01BCN2,100.09
3240382,MDKA05BCN2,103.00
3240383,BWPT01ACN2,100.25


In [10]:
#Tambah accrued interest
bond_list = []
for i in bond['portId'].unique():
    bond_i = bond[bond['portId']==i]
    coupon_i = bond_i['coupon'].unique()[0]*0.01
    outstanding_i = bond_i['outstanding'].unique()[0]
    bond_i['interest'] = (coupon_i*outstanding_i)/365
    bond_i = bond_i.sort_values(by='issueddate')
    bond_i['accruedinterest'] = bond_i['interest'].cumsum()
    bond_list.append(bond_i)
bond = pd.concat(bond_list)
bond = bond.reset_index(drop=True)
bond['marketvalue'] = bond['nominal']*bond['lastPrice']*0.01 + bond['accruedinterest']


In [11]:
sbn = bond[bond['portType']=='S']
corpo = bond[bond['portType']=='B']

In [50]:
example_date = ['2025-02-22','2025-02-23','2025-02-24','2025-02-25','2025-02-26']
example_date = list(sbn['issueddate'].tail(20))


In [70]:
example_date = list(sbn[pd.to_datetime(sbn['issueddate']).apply(lambda x: x.year >= 2019)]['issueddate'])

In [71]:
#Create index
sbn_list = []
for en,i in enumerate(example_date):
    if en<1:
        # print(f"if date: {i}")   
        sbn_i = sbn[sbn['issueddate']==i]
        sbn_i = sbn_i.dropna(subset='marketvalue')
        total_nominal= sbn_i['nominal'].sum()
        total_interest = sbn_i['interest'].sum()
        total_accruedinterest = sbn_i['accruedinterest'].sum()
        total_marketvalue = sbn_i['marketvalue'].sum()
        total_unit = total_marketvalue*0.0001
        nab = total_marketvalue/total_unit
        sbn_df = pd.DataFrame({'Date':i,'Total Nominal':[total_nominal],'Total Interest':[total_interest],'Total Accrued Interest':[total_accruedinterest],
                           'Total Market Value':[total_marketvalue],'Total Unit':[total_unit],'NAB':[nab]},index=[en])
        sbn_list.append(sbn_df)
    else:
        # print(f"else date: {i}")
        sbn_i_t_1 = sbn_list[en-1]
        sbn_i = sbn[sbn['issueddate']==i]
        sbn_i = sbn_i.dropna(subset='marketvalue')
        total_nominal= sbn_i['nominal'].sum()
        total_interest = sbn_i['interest'].sum()
        total_accruedinterest = total_accruedinterest+total_interest
        total_marketvalue = sbn_i['marketvalue'].sum()
        nab = (total_marketvalue - (total_nominal - sbn_i_t_1['Total Nominal'].values[0])) / total_unit
        # print(f"date: {i}, unit: {total_unit:,.2f}, nominal_t: {total_nominal:,.2f}, nominal_(t-1) {sbn_i_t_1['Total Nominal'].values[0]:,.2f}, NAB: {nab:,.2f}")
        total_unit = total_unit+(total_nominal-sbn_i_t_1['Total Nominal'].values[0])/nab
        sbn_df = pd.DataFrame({'Date':i,'Total Nominal':[total_nominal],'Total Interest':[total_interest],'Total Accrued Interest':[total_accruedinterest],
                           'Total Market Value':[total_marketvalue],'Total Unit':[total_unit],'NAB':[nab]},index=[en])
        sbn_list.append(sbn_df)
sbn_index = pd.concat(sbn_list)

In [73]:
sbn_index.to_csv("sbn_index_2019.csv")

In [74]:
sbn_index

,Date,Total Nominal,Total Interest,Total Accrued Interest,Total Market Value,Total Unit,NAB
0,2019-01-01,1.229864e+15,7.061942e+10,1.544049e+14,1.403937e+15,1.403937e+11,1.000000e+04
1,2019-01-02,1.230864e+15,7.061942e+10,1.544755e+14,1.404723e+15,1.404937e+11,9.998480e+03
2,2019-01-03,1.230864e+15,7.061942e+10,1.545461e+14,1.401644e+15,1.404937e+11,9.976561e+03
3,2019-01-04,1.230864e+15,7.061942e+10,1.546168e+14,1.397043e+15,1.404937e+11,9.943814e+03
4,2019-01-05,1.225864e+15,7.061942e+10,1.546874e+14,1.392127e+15,1.399909e+11,9.944409e+03
...,...,...,...,...,...,...,...
279873,2025-02-22,6.288151e+15,3.044113e+11,4.344363e+16,6.659865e+15,3.087921e+08,2.156747e+07
279874,2025-02-23,6.288151e+15,3.044113e+11,4.344393e+16,6.660169e+15,3.087921e+08,2.156845e+07
279875,2025-02-24,6.320540e+15,3.044113e+11,4.344424e+16,6.691691e+15,3.102940e+08,2.156565e+07
279876,2025-02-25,6.320540e+15,3.044113e+11,4.344454e+16,6.669328e+15,3.102940e+08,2.149358e+07


In [14]:
period = '3 Month'
one_month = 21 #days
rolling_dic = {'1 Month':one_month,'3 Month':one_month*3,'6 Month':one_month*6,'1 Year':one_month*12}
sbn_j_list = []
for en,i in enumerate(set(list(sbn['portId']))):
    # print(en)
    sbn_i = sbn[sbn['portId']==i].drop_duplicates()
    sbn_i = sbn_i.sort_values(by='issueddate')
    sbn_i['issueddatedt'] = pd.to_datetime(sbn_i['issueddate'])
    for j in list(sbn_i['issueddate'].unique()):
        sbn_j = sbn_i[sbn_i['issueddate']==j]
        try:
            cd_pos = date_list.index(j)
            pd_pos = cd_pos - rolling_dic[period]
            cd_ = datetime.datetime.strptime(j, '%Y-%m-%d')
            pd_ = datetime.datetime.strptime(date_list[pd_pos], '%Y-%m-%d')
            sbn_i_inside_dates = sbn_i[(sbn_i['issueddatedt']>=pd_)&(sbn_i['issueddatedt']<=cd_)]
            if len(sbn_i_inside_dates)>0:
                sbn_j[f'sumvalueTrans {period}'] = sum(sbn_i_inside_dates['valueTrans'])
                sbn_j[f'avgvalueTrans {period}'] = sum(sbn_i_inside_dates['valueTrans'])/len(sbn_i_inside_dates['valueTrans'])
                #sbn_j[f'pctvalueTrans {period}'] = len(sbn_i_inside_dates['valueTrans'])/(rolling_dic[period]+2)
                sbn_j_list.append(sbn_j)
            else: pass
        except ValueError: pass
sbn_vt = pd.concat(sbn_j_list)
sbn_vt = sbn_vt.reset_index(drop=True)
sbn_vt = sbn_vt.drop(columns='issueddatedt')


In [15]:
sbn_vt

,issueddate,portId,portrank,portrank2,highPrice,loPrice,lastPrice,frekuensi,volumeTrans,valueTrans,WAPPrice,coupon,YTMValue,MDValue,portId2,portName,portType,syariah,dollar,outstanding,jtdate,portissuer,portReleased,periodekupon,flag,issuedyear,tenorday,tenoryear,tradeday,weekend,USDIDR,nominal,interest,accruedinterest,marketvalue,sumvalueTrans 3 Month,avgvalueTrans 3 Month
0,2024-06-07,IDSR070325273S,0,NIG,100.00,100.00,100.00,100.0,100.0,100.000,100.00,7.33499,0.0,0.736964,IDSR070325273S,Sekuritas Rupiah Bank Indonesia IDSR070325273S,S,N,N,1.370000e+12,2025-03-07,BI,2024-06-07,0.0,1.0,2024.0,273.0,0.747945,Y,N,16279.0,1.370000e+12,2.753133e+08,2.753133e+08,1.370275e+12,100.000,100.000000
1,2024-06-10,IDSR070325273S,0,NIG,100.00,100.00,100.00,100.0,100.0,100.000,100.00,7.33499,0.0,0.736964,IDSR070325273S,Sekuritas Rupiah Bank Indonesia IDSR070325273S,S,N,N,1.370000e+12,2025-03-07,BI,2024-06-07,0.0,1.0,2024.0,273.0,0.747945,N,N,16218.0,1.370000e+12,2.753133e+08,1.101253e+09,1.371101e+12,400.000,100.000000
2,2024-06-11,IDSR070325273S,0,NIG,100.00,100.00,100.00,100.0,100.0,100.000,100.00,7.33499,0.0,0.736964,IDSR070325273S,Sekuritas Rupiah Bank Indonesia IDSR070325273S,S,N,N,1.370000e+12,2025-03-07,BI,2024-06-07,0.0,1.0,2024.0,273.0,0.747945,N,N,16290.0,1.370000e+12,2.753133e+08,1.376567e+09,1.371377e+12,500.000,100.000000
3,2024-06-12,IDSR070325273S,0,NIG,100.00,100.00,100.00,100.0,100.0,100.000,100.00,7.33499,0.0,0.736964,IDSR070325273S,Sekuritas Rupiah Bank Indonesia IDSR070325273S,S,N,N,1.370000e+12,2025-03-07,BI,2024-06-07,0.0,1.0,2024.0,273.0,0.747945,N,N,16295.0,1.370000e+12,2.753133e+08,1.651880e+09,1.371652e+12,600.000,100.000000
4,2024-06-13,IDSR070325273S,0,NIG,100.00,100.00,100.00,100.0,100.0,100.000,100.00,7.33499,0.0,0.736964,IDSR070325273S,Sekuritas Rupiah Bank Indonesia IDSR070325273S,S,N,N,1.370000e+12,2025-03-07,BI,2024-06-07,0.0,1.0,2024.0,273.0,0.747945,N,N,16297.0,1.370000e+12,2.753133e+08,1.927193e+09,1.371927e+12,700.000,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332812,2024-02-05,SPN03240214,-,-,99.12,99.12,99.12,1.0,20.0,19.824,99.12,6.00000,6.041302,0.145074,SPN03240214,Surat Perbendaharaan Negara Seri SPN03240214,S,N,N,4.000000e+10,2024-02-14,NKRI,2023-11-17,4.0,1.0,2023.0,89.0,0.243836,N,N,15688.0,4.000000e+10,6.575342e+06,3.156164e+08,3.996362e+10,971.372,20.236917
332813,2024-02-06,SPN03240214,-,-,99.12,99.12,99.12,1.0,20.0,19.824,99.12,6.00000,6.041302,0.145074,SPN03240214,Surat Perbendaharaan Negara Seri SPN03240214,S,N,N,4.000000e+10,2024-02-14,NKRI,2023-11-17,4.0,1.0,2023.0,89.0,0.243836,N,N,15705.0,4.000000e+10,6.575342e+06,3.221918e+08,3.997019e+10,991.196,20.228490
332814,2024-02-07,SPN03240214,-,-,99.12,99.12,99.12,1.0,20.0,19.824,99.12,6.00000,6.041302,0.145074,SPN03240214,Surat Perbendaharaan Negara Seri SPN03240214,S,N,N,4.000000e+10,2024-02-14,NKRI,2023-11-17,4.0,1.0,2023.0,89.0,0.243836,N,N,15734.0,4.000000e+10,6.575342e+06,3.287671e+08,3.997677e+10,1011.020,20.220400
332815,2024-02-12,SPN03240214,-,-,99.12,99.12,99.12,1.0,20.0,19.824,99.12,6.00000,6.041302,0.145074,SPN03240214,Surat Perbendaharaan Negara Seri SPN03240214,S,N,N,4.000000e+10,2024-02-14,NKRI,2023-11-17,4.0,1.0,2023.0,89.0,0.243836,N,N,15685.0,4.000000e+10,6.575342e+06,3.616438e+08,4.000964e+10,1110.140,20.184364


In [ ]:
reb_date = ['2025-02-03','2024-10-01','2024-07-01','2024-04-01']

In [ ]:
# Market Value - Accrued Interest
# Pietro Veronesi - Fixed Income Securities_ Valuation, Risk, and Risk Management (2010, Wiley)
# Chapter 2.4, 2.4.4.2 Treasury Coupon Notes and Bonds page 51

with pd.ExcelWriter(f'obligasi_negara_tanpa_min_jatuh_tempo_0_MV_new.xlsx') as writer:
    for date_ in reb_date:
        sbn_1 = sbn_vt_df.copy()
        sbn_1['marketvalue'] = sbn_1['lastPrice']*sbn_1['outstanding adj'] # Dikurangin accrual
        sbn_1['date'] = date_
        sbn_1['date'] = pd.to_datetime(sbn_1['date'])
        sbn_1['jtdatedt'] = pd.to_datetime(sbn_1['jtdate'])
        sbn_1['issueddatedt'] = pd.to_datetime(sbn_1['issueddate'])
        sbn_1 = sbn_1[sbn_1['jtdatedt']>date_]
        sbn_1 = sbn_1[sbn_1['issueddatedt']<date_]
        cd_pos = date_list.index(date_)
        pd_pos = cd_pos - rolling_dic[period]
        cd_ = datetime.datetime.strptime(date_, '%Y-%m-%d')
        pd_ = datetime.datetime.strptime(date_list[pd_pos], '%Y-%m-%d')
        sbn_1 = sbn_1[(sbn_1['issueddatedt']>=pd_)&(sbn_1['issueddatedt']<=cd_)]
        
        # Market Value - Accrued Interest
        coupon = sbn_1['coupon'] / 100
        outstanding = sbn_1['outstanding adj']
        periodekupon = sbn_1['periodekupon']  # Periode Kupon per tahun
        tenorday = sbn_1['tenorday']  # Umur Obligasi
        jtdate = sbn_1['jtdatedt']  # Tanggal Maturity
        valid_condition = (tenorday > 0.0) & (periodekupon >= 1.0) & (tenorday < 1e10) & (periodekupon < 1e15) & (outstanding > 0.0)
        coupon_interval = tenorday / periodekupon
        coupon_interval[~valid_condition] = None  # Set NaN, -inf, inf to None
        last_coupon_date = jtdate - pd.to_timedelta(coupon_interval, unit='D')
        days_since_last_coupon = abs((pd.to_datetime('today') - last_coupon_date).dt.days)
        # print((coupon == 0).sum(), (outstanding == 0).sum(), (periodekupon == 0).sum(), (days_since_last_coupon == 0).sum(), (coupon_interval == 0).sum())
        sbn_1['accrued_interest'] = (coupon * outstanding / periodekupon) * (days_since_last_coupon / coupon_interval) # Calculate accrued interest
        sbn_1['marketvalue_accruedinterest'] = (sbn_1['lastPrice'] * sbn_1['outstanding adj']) - sbn_1['accrued_interest'] # Calculate market value

        sbn_1 = sbn_1[['portId', 'portName', 'date','issueddate',
                    'portReleased','jtdate','avgvalueTrans 3 Month','lastPrice',
                    'outstanding adj','marketvalue', 'marketvalue_accruedinterest', 'accrued_interest']]
        
        sbn_1 = sbn_1.drop_duplicates(subset='portId',keep='last')
        sbn_1 = sbn_1.dropna(subset='marketvalue')
        sbn_1 = sbn_1[sbn_1['marketvalue']!=0]
        sbn_1['MV Weight'] = sbn_1['marketvalue']/sum(sbn_1['marketvalue'])
        sbn_1 = sbn_1.dropna(subset='marketvalue_accruedinterest')
        sbn_1 = sbn_1[sbn_1['marketvalue_accruedinterest']!=0]
        sbn_1['MV - AI Weight'] = sbn_1['marketvalue_accruedinterest']/sum(sbn_1['marketvalue_accruedinterest'])
        sbn_1 = sbn_1.rename(columns={'portId':'ID','portName':'Nama Obligasi','date':'Rebalancing Date',
                                    'issueddate':'Last Transaction Date','portReleased':'Issuance Date',
                                    'jtdate':'Jatuh Tempo','outstanding adj':'Nilai Outstanding','avgvalueTrans 3 Month':'Average Transaction Value 3 Month',
                                    'lastPrice':'Last Price','marketvalue':'Market Value', 'marketvalue_accruedinterest': 'MV - AI', 'accrued_interest': 'AI'})
        sbn_1['Rebalancing Date'] = sbn_1['Rebalancing Date'].astype('str')
        sbn_1 = sbn_1.sort_values(by='Average Transaction Value 3 Month',ascending=False)
        # sbn_1 = sbn_1.head(50) #Ambil 50 bond
        sbn_1 = sbn_1.reset_index(drop=True)
        sbn_1.to_excel(writer, sheet_name=f'{date_}')

sbn_1

In [ ]:
# Market Value - Accrued Interest (Excel IGBI 2025)

with pd.ExcelWriter(f'obligasi_negara_tanpa_min_jatuh_tempo_0_MV_new.xlsx') as writer:
    for date_ in reb_date:
        sbn_1 = sbn_vt_df.copy()
        sbn_1['marketvalue'] = sbn_1['lastPrice']*sbn_1['outstanding adj'] # Dikurangin accrual
        sbn_1['date'] = date_
        sbn_1['date'] = pd.to_datetime(sbn_1['date'])
        sbn_1['jtdatedt'] = pd.to_datetime(sbn_1['jtdate'])
        sbn_1['issueddatedt'] = pd.to_datetime(sbn_1['issueddate'])
        sbn_1 = sbn_1[sbn_1['jtdatedt']>date_]
        sbn_1 = sbn_1[sbn_1['issueddatedt']<date_]
        cd_pos = date_list.index(date_)
        pd_pos = cd_pos - rolling_dic[period]
        cd_ = datetime.datetime.strptime(date_, '%Y-%m-%d')
        pd_ = datetime.datetime.strptime(date_list[pd_pos], '%Y-%m-%d')
        sbn_1 = sbn_1[(sbn_1['issueddatedt']>=pd_)&(sbn_1['issueddatedt']<=cd_)]
        
        # Market Value - Accrued Interest
        coupon = sbn_1['coupon']/100
        outstanding = sbn_1['outstanding adj']
        ytm = sbn_1['YTMValue']/100 + 1
        valid_condition = (coupon > 0.0) & (outstanding > 0.0) & (ytm > 0.0)
        condition_check = coupon * ytm * outstanding
        condition_check[~valid_condition] = None  # Set NaN, -inf, inf to None
        
        # print((coupon == 0).sum(), (outstanding == 0).sum(), (ytm == 0).sum())
        sbn_1['accrued_interest'] = coupon * ytm * outstanding / 365 # Calculate accrued interest
        sbn_1['marketvalue_accruedinterest'] = (sbn_1['lastPrice'] * sbn_1['outstanding adj']) - sbn_1['accrued_interest'] # Calculate market value

        sbn_1 = sbn_1[['portId', 'portName', 'date','issueddate',
                    'portReleased','jtdate','avgvalueTrans 3 Month','lastPrice',
                    'outstanding adj','marketvalue', 'marketvalue_accruedinterest', 'accrued_interest']]
        
        sbn_1 = sbn_1.drop_duplicates(subset='portId',keep='last')
        sbn_1 = sbn_1.dropna(subset='marketvalue')
        sbn_1 = sbn_1[sbn_1['marketvalue']!=0]
        sbn_1['MV Weight'] = sbn_1['marketvalue']/sum(sbn_1['marketvalue'])
        sbn_1 = sbn_1.dropna(subset='marketvalue_accruedinterest')
        sbn_1 = sbn_1[sbn_1['marketvalue_accruedinterest']!=0]
        sbn_1['MV - AI Weight'] = sbn_1['marketvalue_accruedinterest']/sum(sbn_1['marketvalue_accruedinterest'])
        sbn_1 = sbn_1.rename(columns={'portId':'ID','portName':'Nama Obligasi','date':'Rebalancing Date',
                                    'issueddate':'Last Transaction Date','portReleased':'Issuance Date',
                                    'jtdate':'Jatuh Tempo','outstanding adj':'Nilai Outstanding','avgvalueTrans 3 Month':'Average Transaction Value 3 Month',
                                    'lastPrice':'Last Price','marketvalue':'Market Value', 'marketvalue_accruedinterest': 'MV - AI', 'accrued_interest': 'AI'})
        sbn_1['Rebalancing Date'] = sbn_1['Rebalancing Date'].astype('str')
        sbn_1 = sbn_1.sort_values(by='Average Transaction Value 3 Month',ascending=False)
        # sbn_1 = sbn_1.head(50) #Ambil 50 bond
        sbn_1 = sbn_1.reset_index(drop=True)
        sbn_1.to_excel(writer, sheet_name=f'{date_}')

sbn_1

In [ ]:
with pd.ExcelWriter(f'obligasi_negara_tanpa_min_jatuh_tempo_0.xlsx') as writer:
    for date_ in reb_date:
        sbn_1 = sbn_vt.copy()
        sbn_1['marketvalue'] = sbn_1['lastPrice']*sbn_1['outstanding'] # Dikurangin accrual
        # sbn_1['accruaedinterest'] = 
        sbn_1['date'] = date_
        sbn_1['date'] = pd.to_datetime(sbn_1['date'])
        sbn_1['jtdatedt'] = pd.to_datetime(sbn_1['jtdate'])
        sbn_1['issueddatedt'] = pd.to_datetime(sbn_1['issueddate'])
        sbn_1 = sbn_1[sbn_1['jtdatedt']>date_]
        sbn_1 = sbn_1[sbn_1['issueddatedt']<date_]
        cd_pos = date_list.index(date_)
        pd_pos = cd_pos - rolling_dic[period]
        cd_ = datetime.datetime.strptime(date_, '%Y-%m-%d')
        pd_ = datetime.datetime.strptime(date_list[pd_pos], '%Y-%m-%d')
        sbn_1 = sbn_1[(sbn_1['issueddatedt']>=pd_)&(sbn_1['issueddatedt']<=cd_)]
        sbn_1 = sbn_1[['portId','portName','date','issueddate','portReleased','jtdate','avgvalueTrans 3 Month','lastPrice','outstanding','marketvalue']]
        sbn_1 = sbn_1.drop_duplicates(subset='portId',keep='last')
        sbn_1 = sbn_1.dropna(subset='marketvalue')
        sbn_1 = sbn_1[sbn_1['marketvalue']!=0]
        sbn_1['Weight'] = sbn_1['marketvalue']/sum(sbn_1['marketvalue'])
        sbn_1 = sbn_1.rename(columns={'portId':'ID','portName':'Nama Obligasi','date':'Rebalancing Date','issueddate':'Last Transaction Date','portReleased':'Issuance Date','jtdate':'Jatuh Tempo','outstanding':'Nilai Outstanding','avgvalueTrans 3 Month':'Average Transaction Value 3 Month','lastPrice':'Last Price','marketvalue':'Market Value'})
        sbn_1['Rebalancing Date'] = sbn_1['Rebalancing Date'].astype('str')
        sbn_1 = sbn_1.sort_values(by='Average Transaction Value 3 Month',ascending=False)
        sbn_1 = sbn_1.head(50) #Ambil 50 bond
        sbn_1 = sbn_1.reset_index(drop=True)
        sbn_1.to_excel(writer, sheet_name=f'{date_}')